In [3]:
from utils.parsing import parse_snp500
from utils.data import TradingDataset, show_data

from torch.utils.data import DataLoader

# Устанавливаем гиперпараметры

In [5]:
MAX_LENGHT = 256
TEST_LENGHT = 32
SPLIT_COEFF = 0.1
BATCH_SIZE = 16

# Парсим данные snp500 в папку "data/" если их нет

In [ ]:
# парсим данные за последние 7+ лет (выборка бOльшего размера просто не влезет в память)
parse_snp500(dirpath='data/', 
             start='2017-01-01', 
             timeframe='1d', 
             max_train_len=256, 
             test_len=32, 
             split_coeff=0.1)

Symbol MMM parse failed!
Symbol AOS parse failed!
Symbol ABT parse failed!
Symbol ABBV parse failed!


# Создаём loader'ы для train и val выборок

In [ ]:
# Получаем датасеты данных для обучения и валидации
train_dataset = TradingDataset(root_dir="data/train", train_len=256, test_len=32)
val_dataset = TradingDataset(root_dir="data/val", train_len=256, test_len=32)

print(f"Train dataset lenght: {len(train_dataset)}\nValidation dataset lenght: {len(val_dataset)}\n")

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"Train loader lenght: {len(train_loader)}\nValidation loader lenght: {len(val_loader)}")

# Посмотрим на то что хранится в loader'ах

In [ ]:
show_data(val_loader)

# Построим модель

In [ ]:
%run models/VGG16Based.py

myModel = VGG16Based(seq_len=60)
myModel

In [ ]:
train_sample = next(iter(train_loader))[0]
myModel(train_sample).shape, next(iter(train_loader))[1].shape

In [1]:
import os
import matplotlib.pyplot as plt

def show_quality (model, loader, num=4):
    train, test = next(iter(loader))
    
    train = train[:num]
    test = test[:num]
    
    pred = model(train).detach()

    fig, axs = plt.subplots(int(num**(1/2)), int(num**(1/2)), figsize=(20, 10))
    
    for i in range(num):
        row = i // int(num**(1/2))
        col = i % int(num**(1/2))

        real = torch.hstack((train[i, 3, :], test[i, 0, :]))
        predict = torch.hstack((train[i, 3, :], pred[i, 0, :]))

        axs[row, col].plot(predict, "red")
        axs[row, col].plot(real, "black")
        axs[row, col].grid()
    
    image_path = 'plot_image.png'
    plt.savefig(image_path)
    plt.show()

    os.remove(image_path)

In [2]:
show_quality(myModel, val_loader)

NameError: name 'myModel' is not defined